# 下載前的準備

In [1]:
#如果沒有此套件, 則將 下面這行↓ 的註解去掉(將井字號去掉即可)
#!pip install --user webdriver_manager

In [1]:
# 引入套件
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re
import os
from selenium.webdriver import ActionChains
from time import sleep
from collections import Counter

In [2]:
# 定義函式
def get_report(url): #下載report的func
    sub_page = webdriver.Chrome()
    sub_page.get(url)
    soup = BeautifulSoup(sub_page.page_source, 'lxml')
    sleep(1)
    logo = soup.find('img')

    if(logo['src'] == '/images/sealTop.gif'):
        download_temp = sub_page.find_element_by_css_selector('div > table > tbody > tr:nth-child(2) > td:nth-child(3) > a').get_attribute('href')
        if(download_temp[19:27] == '/ix?doc='):
            download_address = 'https://www.sec.gov/' + download_temp[28:]
        else:
            download_address = download_temp
        
        ###獲取檔案名稱

        #filing type
        filing_type = sub_page.find_element_by_css_selector('#formName > strong').text
        filing_type = filing_type[5:9]

        #filing date
        filing_date = sub_page.find_element_by_css_selector('div.formContent > div:nth-child(1) > div:nth-child(2)').text


        #公司名
        company_temp = soup.find('div', {'class': 'companyInfo'}).next_element.next_element.next_element
        company_name = company_temp[:-10]
        forbidden = re.compile('[\\\/:*?"<>|]')
        company_name = re.sub(forbidden, ' ', company_name)

        #年度、季度
        date = sub_page.find_element_by_css_selector('div.formContent > div:nth-child(2) > div.info').text
        fiscal_year = date[0:4] 
        month = date[5:7].strip('0')
        fiscal_period = get_period(month)

        #名稱合併
        filename = company_name + ' _ ' + fiscal_year + ' _ ' + fiscal_period + ' _ ' + filing_date + '(' + filing_type + ')'
        if(filename == 'AMERICAN PHYSICIANS SERVICE GROUP INC _ 2000 _ Q1 _ 2001-03-23(10-Q)'): #之前在試跑時, 此公司的此檔案無法取得, 故直接跳過
            return
        
        #將檔案存進叫All_Report的資料夾內
        home = os.getcwd()
        path = home + '\\' + 'All_Report'
        if not os.path.exists(path):
            os.mkdir(path)
        os.chdir(path)
        headers = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
        }
        if(download_address[-3:] == 'txt'):
            response = requests.get(download_address, headers=headers)
            with open(filename+".txt", 'w', encoding='UTF-8') as file:
                file.write(response.text)
                file.close()
        else:
            response = requests.get(download_address, headers=headers)
            with open(filename+".htm", 'wb') as file:
                file.write(response.content)
                file.close()

        os.chdir(home)

        #關閉瀏覽器
        sub_page.close()
    else:
        return
def get_period(month): #取得季度的func
    month = int(month)
    if(month >= 10):
        return "Q4"
    elif(month >= 7):
        return "Q3"
    elif(month >= 4):
        return "Q2"
    else:
        return "Q1"

# 開始下載

In [3]:
#美國所有洲的代碼
states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'X1', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

#用一周作為一個時間段落間隔, 因為此網頁搜尋結果最多只能呈現十頁, 且每頁一百筆, 故用這種方法盡可能地讓搜尋結果縮小在1000筆以下, 獲取所有報表
#但若是在下載過程中, 有些州在該週的報表數仍過多, 就只能在時間效率和資料完整之間選擇一個。

dates = []
for year in range(2001, 2022):
    for month in range(1,13):
        if(month == 1 or month == 3 or month == 5 or month == 7 or month == 8 or month == 10 or month == 12): #大月
            if(month < 10):
                month = '0' + str(month)
            dates.append(str(year) + '-' + str(month) + '-01')
            dates.append(str(year) + '-' + str(month) + '-07')
            dates.append(str(year) + '-' + str(month) + '-08')
            dates.append(str(year) + '-' + str(month) + '-15')
            dates.append(str(year) + '-' + str(month) + '-16')
            dates.append(str(year) + '-' + str(month) + '-23')
            dates.append(str(year) + '-' + str(month) + '-24')
            dates.append(str(year) + '-' + str(month) + '-31')
        elif(month == 4 or month == 6 or month == 9 or month == 11):                                         #小月
            if(month < 10):
                month = '0' + str(month)
            dates.append(str(year) + '-' + str(month) + '-01')
            dates.append(str(year) + '-' + str(month) + '-07')
            dates.append(str(year) + '-' + str(month) + '-08')
            dates.append(str(year) + '-' + str(month) + '-15')
            dates.append(str(year) + '-' + str(month) + '-16')
            dates.append(str(year) + '-' + str(month) + '-23')
            dates.append(str(year) + '-' + str(month) + '-24')
            dates.append(str(year) + '-' + str(month) + '-30')
        else:
            if((((year) % 4) == 0 and ((year) % 100) != 0) or ((year) % 400) == 0): #閏年
                if(month < 10):
                    month = '0' + str(month)
                dates.append(str(year) + '-' + str(month) + '-01')
                dates.append(str(year) + '-' + str(month) + '-07')
                dates.append(str(year) + '-' + str(month) + '-08')
                dates.append(str(year) + '-' + str(month) + '-15')
                dates.append(str(year) + '-' + str(month) + '-16')
                dates.append(str(year) + '-' + str(month) + '-23')
                dates.append(str(year) + '-' + str(month) + '-24')
                dates.append(str(year) + '-' + str(month) + '-29')
            else:                                                                   #非閏年
                if(month < 10):
                    month = '0' + str(month)
                dates.append(str(year) + '-' + str(month) + '-01')
                dates.append(str(year) + '-' + str(month) + '-07')
                dates.append(str(year) + '-' + str(month) + '-08')
                dates.append(str(year) + '-' + str(month) + '-15')
                dates.append(str(year) + '-' + str(month) + '-16')
                dates.append(str(year) + '-' + str(month) + '-23')
                dates.append(str(year) + '-' + str(month) + '-24')
                dates.append(str(year) + '-' + str(month) + '-28')

In [4]:
# 這段程式碼執行後可以看到所有的時間區段
for i in range(0, len(dates), 2):
    print(dates[i], dates[i+1])

2001-01-01 2001-01-07
2001-01-08 2001-01-15
2001-01-16 2001-01-23
2001-01-24 2001-01-31
2001-02-01 2001-02-07
2001-02-08 2001-02-15
2001-02-16 2001-02-23
2001-02-24 2001-02-28
2001-03-01 2001-03-07
2001-03-08 2001-03-15
2001-03-16 2001-03-23
2001-03-24 2001-03-31
2001-04-01 2001-04-07
2001-04-08 2001-04-15
2001-04-16 2001-04-23
2001-04-24 2001-04-30
2001-05-01 2001-05-07
2001-05-08 2001-05-15
2001-05-16 2001-05-23
2001-05-24 2001-05-31
2001-06-01 2001-06-07
2001-06-08 2001-06-15
2001-06-16 2001-06-23
2001-06-24 2001-06-30
2001-07-01 2001-07-07
2001-07-08 2001-07-15
2001-07-16 2001-07-23
2001-07-24 2001-07-31
2001-08-01 2001-08-07
2001-08-08 2001-08-15
2001-08-16 2001-08-23
2001-08-24 2001-08-31
2001-09-01 2001-09-07
2001-09-08 2001-09-15
2001-09-16 2001-09-23
2001-09-24 2001-09-30
2001-10-01 2001-10-07
2001-10-08 2001-10-15
2001-10-16 2001-10-23
2001-10-24 2001-10-31
2001-11-01 2001-11-07
2001-11-08 2001-11-15
2001-11-16 2001-11-23
2001-11-24 2001-11-30
2001-12-01 2001-12-07
2001-12-08

2015-07-08 2015-07-15
2015-07-16 2015-07-23
2015-07-24 2015-07-31
2015-08-01 2015-08-07
2015-08-08 2015-08-15
2015-08-16 2015-08-23
2015-08-24 2015-08-31
2015-09-01 2015-09-07
2015-09-08 2015-09-15
2015-09-16 2015-09-23
2015-09-24 2015-09-30
2015-10-01 2015-10-07
2015-10-08 2015-10-15
2015-10-16 2015-10-23
2015-10-24 2015-10-31
2015-11-01 2015-11-07
2015-11-08 2015-11-15
2015-11-16 2015-11-23
2015-11-24 2015-11-30
2015-12-01 2015-12-07
2015-12-08 2015-12-15
2015-12-16 2015-12-23
2015-12-24 2015-12-31
2016-01-01 2016-01-07
2016-01-08 2016-01-15
2016-01-16 2016-01-23
2016-01-24 2016-01-31
2016-02-01 2016-02-07
2016-02-08 2016-02-15
2016-02-16 2016-02-23
2016-02-24 2016-02-29
2016-03-01 2016-03-07
2016-03-08 2016-03-15
2016-03-16 2016-03-23
2016-03-24 2016-03-31
2016-04-01 2016-04-07
2016-04-08 2016-04-15
2016-04-16 2016-04-23
2016-04-24 2016-04-30
2016-05-01 2016-05-07
2016-05-08 2016-05-15
2016-05-16 2016-05-23
2016-05-24 2016-05-31
2016-06-01 2016-06-07
2016-06-08 2016-06-15
2016-06-16

# 10-Q Report 季報

In [11]:
for i in range(0, len(dates), 2):
    #斷點1
    time_index = 0        #當程式隔天重啟時使用, 假設上次time index最後顯示在 2, 則將這行的數字改為2, default 0, 且只會是偶數
    if(i >= time_index):  #當time_index被改成
        print('Current time index: ', i, ';time period', dates[i], 'to', dates[i+1])
        for j in range(0, len(states)):
            #斷點2
            if(i == time_index):
                state_index = 0 #當程式因為網路不穩..等情況跳掉時使用, 假設上次state index最後顯示在 4, 則將這行的數字改為4, default 0
            else:
                state_index = 0
                
            if(j >= state_index):
                print('Current State index: ', j, 'state', states[j])
                driver = webdriver.Chrome()
                link = 'https://www.sec.gov/edgar/search/#/dateRange=custom&category=custom&locationType=incorporated&locationCode=' + states[j] + '&startdt=' + dates[i] + '&enddt=' + dates[i+1] + '&forms=10-Q'
                driver.get(link)
                driver.refresh()
                sleep(3)

                exist = driver.find_element_by_css_selector('#no-results-grid > div').get_attribute("style")
                sleep(1)
                if(exist == 'display: none;'):
                    soup = BeautifulSoup(driver.page_source, 'lxml')
                    sleep(1)

                    pages = driver.find_element_by_css_selector('#show-result-count > h5').text
                    sleep(2)
                #計算結果共有幾頁, 若大於10頁便會列印出是哪一州哪一個時間區段, 有呈現出來的1000筆資料仍會進行下載, 而多於1000筆的不會被下載到
                    pages = re.sub(',', '', pages)
                    if(int(pages[:-15])%100 != 0):
                        pages = int(int(pages[:-15])/100)+1
                    else: 
                        pages = int(int(pages[:-15])/100)
                    if(pages > 10):
                        print('state', states[j], 'in period' , dates[i], '~', dates[i+1], 'has more than 1,000 results')

                    #下載
                    for page in range(1, pages+1):
                        if(page > 10): #最大只會有十頁(1000筆報表)
                            break
                            
                        #斷點3
                        if(i == time_index and j == state_index):
                            page_index = 1   #假設上次下載到第5頁, 則將這行的數字改為 5, default 1
                        else:
                            page_index = 1
                            
                        #下載各頁的報表
                        if(page >= page_index):
                            files = driver.find_elements_by_css_selector("a[class='preview-file']" )
                            #斷點4
                            if(i == time_index and j == state_index and page == page_index):
                                file_index = 0   #假設上次只顯示到第5頁第3個檔案, 則將這行的數字改為 3, default 0
                            else:
                                file_index = 0

                            for k in (file_index, len(files)):
                                file = files[k]
                                print('Download staus: page', page, 'no.', k, 'file')
                                ActionChains(driver).move_to_element(file).click().perform()
                                sleep(2)
                                soup = BeautifulSoup(driver.page_source, 'lxml')
                                sleep(1)
                                link = soup.find('a', {'id' : 'open-submission'})
                                link = link['href']
                                close = driver.find_element_by_css_selector('#previewer > div > div > div.modal-footer.border.border-0 > button')
                                ActionChains(driver).move_to_element(close).click().perform()
                                get_report(link)
                        #換頁功能    
                        if(page != pages):
                            next_page = page+1
                            link = 'https://www.sec.gov/edgar/search/#/dateRange=custom&category=custom&locationType=incorporated&locationCode=' + states[j] + '&startdt=' + dates[i] + '&enddt=' + dates[i+1] + '&forms=10-Q&page=' + str(next_page)
                            driver.get(link)
                            driver.refresh()
                            sleep(3)
                    driver.close()
print('10Q report 全部下載完成')

10Q report 全部下載完成


# 10-K Report 年報

In [19]:
for i in range(0, len(dates), 2):
    #斷點1
    time_index = 0        #當程式隔天重啟時使用, 假設上次time index最後顯示在 2, 則將這行的數字改為2, default 0, 且只會是偶數
    if(i >= time_index):  #當time_index被改成
        print('Current time index: ', i, '; time period', dates[i], 'to', dates[i+1])
        for j in range(0, len(states)):
            #斷點2
            if(i == time_index):
                state_index = 0 #當程式因為網路不穩..等情況跳掉時使用, 假設上次state index最後顯示在 4, 則將這行的數字改為4, default 0
            else:
                state_index = 0
                
            if(j >= state_index):
                print('Current State index: ', j, 'state', states[j])
                driver = webdriver.Chrome()
                link = 'https://www.sec.gov/edgar/search/#/dateRange=custom&category=custom&locationType=incorporated&locationCode=' + states[j] + '&startdt=' + dates[i] + '&enddt=' + dates[i+1] + '&forms=10-K'
                driver.get(link)
                sleep(1)

                exist = driver.find_element_by_css_selector('#no-results-grid > div').get_attribute("style")
                sleep(1)
                if(exist == 'display: none;'):
                    soup = BeautifulSoup(driver.page_source, 'lxml')
                    sleep(1)

                    pages = driver.find_element_by_css_selector('#show-result-count > h5').text

                    #計算結果共有幾頁, 若大於10頁便會列印出是哪一州哪一個時間區段, 有呈現出來的1000筆資料仍會進行下載, 而多於1000筆的不會被下載到
                    pages = re.sub(',', '', pages)
                    if(int(pages[:-15])%100 != 0):
                        pages = int(int(pages[:-15])/100)+1
                    else: 
                        pages = int(int(pages[:-15])/100)
                    if(pages > 10):
                        print('state', states[j], 'in period' , dates[i], '~', dates[i+1], 'has more than 1,000 results')

                    #下載
                    for page in range(1, pages+1):
                        if(page > 10): #最大只會有十頁(1000筆報表)
                            break
                            
                        #斷點3
                        if(i == time_index and j == state_index):
                            page_index = 1   #假設上次下載到第5頁, 則將這行的數字改為 5, default 1
                        else:
                            page_index = 1
                            
                        #下載各頁的報表
                        if(page >= page_index):
                            files = driver.find_elements_by_css_selector("a[class='preview-file']" )
                            #斷點4
                            if(i == time_index and j == state_index and page == page_index):
                                file_index = 0   #假設上次只顯示到第5頁第3個檔案, 則將這行的數字改為 3, default 0
                            else:
                                file_index = 0
                            
                            for k in range(file_index, len(files)):
                                file = files[k]
                                print('Download status: page', page, 'no.', k, 'file')
                                ActionChains(driver).move_to_element(file).click().perform()
                                sleep(2)
                                soup = BeautifulSoup(driver.page_source, 'lxml')
                                sleep(1)
                                link = soup.find('a', {'id' : 'open-submission'})
                                link = link['href']
                                close = driver.find_element_by_css_selector('#previewer > div > div > div.modal-footer.border.border-0 > button')
                                ActionChains(driver).move_to_element(close).click().perform()
                                get_report(link)
                        #換頁功能    
                        if(page != pages):
                            next_page = page+1
                            link = 'https://www.sec.gov/edgar/search/#/dateRange=custom&category=custom&locationType=incorporated&locationCode=' + states[j] + '&startdt=' + dates[i] + '&enddt=' + dates[i+1] + '&forms=10-K&page=' + str(next_page)
                            driver.get(link)
                            driver.refresh()
                            sleep(3)
                driver.close()
print('10K report 全部下載完成')

Current time index:  0 ; time period 2001-01-01 to 2001-01-07
Current State index:  21 state MA
Download status: page 1 no. 1 file
Current State index:  22 state MI
Current State index:  23 state MN
Current State index:  24 state MS
Current State index:  25 state MO
Current State index:  26 state MT
Current State index:  27 state NE
Current State index:  28 state NV
Download status: page 1 no. 0 file
Current State index:  29 state NH
Current State index:  30 state NJ
Current State index:  31 state NM
Current State index:  32 state NY
Current State index:  33 state NC
Current State index:  34 state ND
Current State index:  35 state OH
Current State index:  36 state OK
Current State index:  37 state OR
Current State index:  38 state PA
Current State index:  39 state RI
Current State index:  40 state SC
Current State index:  41 state SD


WebDriverException: Message: chrome not reachable
  (Session info: chrome=103.0.5060.134)
